This code is used to resample a collection of GeoTIFF files

### 0. Libraries

In [1]:
import os
import rasterio
from rasterio.warp import reproject, Resampling, calculate_default_transform
from pyproj import CRS

### 1. Resampling


In [ ]:
# Set the input and output folder directories
folder1 = "data/Meteoblue/Air_temperature_Bologna/Estati_22_23/1"    
folder2 = "data/Meteoblue/Air_temperature_Bologna/Estati_22_23/2"
resampled_folder2 = "data/Meteoblue/Air_temperature_Bologna/Estati_22_23/2_resampled"
resampled_folder1 ="data/Meteoblue/Air_temperature_Bologna/Estati_22_23/1_resampled"

In [19]:
# List of GeoTIFF files in folder1
folder1_files = os.listdir(folder1)

# List of GeoTIFF files in folder2
folder2_files = os.listdir(folder2)

#### metodo 1

In [6]:
# Define the resampling method
resampling_method = Resampling.bilinear  


In [13]:
# Loop through the GeoTIFF files in folder2
for folder2_file in folder2_files:
    if folder2_file.endswith('.tif') or folder2_file.endswith('.tiff'):
        # Construct the full paths to input and output files
        input_path = os.path.join(folder2, folder2_file)
        output_path = os.path.join(resampled_folder1, folder2_file)

        # Open the input GeoTIFF file using rasterio
        with rasterio.open(input_path) as src:
            # Get the dimensions of the pixel from the first file in folder1
            reference_file = os.path.join(folder1, folder1_files[0])  # Take the first file from folder1
            with rasterio.open(reference_file) as reference_src:
                # Resample data to match the pixel dimensions from folder1
                data = src.read(
                    out_shape=(
                        src.count,
                        reference_src.height,
                        reference_src.width
                    ),
                    resampling=resampling_method
                )

                # Update the profile with the new dimensions and transform
                profile = src.profile
                profile.update(width=reference_src.width, height=reference_src.height, transform=reference_src.transform)

                # Save the resampled GeoTIFF
                with rasterio.open(output_path, 'w', **profile) as dst:
                    dst.write(data)

#### metodo 2

In [20]:
# Read the pixel dimension of the first file in folder1
with rasterio.open(os.path.join(folder1, os.listdir(folder1)[0])) as reference_src:
    reference_width = reference_src.width
    reference_height = reference_src.height


In [21]:
# Loop through the GeoTIFF files in folder2
for folder2_file in os.listdir(folder2):
    if folder2_file.endswith('.tif') or folder2_file.endswith('.tiff'):
        input_path = os.path.join(folder2, folder2_file)
        output_path = os.path.join(resampled_folder2, folder2_file)

        with rasterio.open(input_path) as src:
            # Calculates scale factors for width and height
            width_scale = reference_width / src.width
            height_scale = reference_height / src.height

            # Define the resampling method
            resampling_method = Resampling.bilinear  

            # Resampling with scaling factors
            data = src.read(
                out_shape=(
                    src.count,
                    int(src.height * height_scale),
                    int(src.width * width_scale)
                ),
                resampling=resampling_method
            )

            # Update profile with new dimensions and transformation
            profile = src.profile
            profile.update(width=int(src.width * width_scale), height=int(src.height * height_scale))

            # Save the resampled file
            with rasterio.open(output_path, 'w', **profile) as dst:
                dst.write(data)
